In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import explode, min, max, col, round, hour, minute, count, dayofmonth, month, substring, avg
from pyspark.sql.types import IntegerType
from pyspark.sql.types import TimestampType
import pandas as pd
import numpy as np
import seaborn as sns

warehouse_location = 'hdfs://namenode:8020/warehouse'
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .enableHiveSupport() \
    .getOrCreate()

22/01/21 13:01:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/21 13:01:34 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/01/21 13:01:34 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
df_telemetria = spark.read.json('hdfs://namenode:8020//raw/telemetria/b305d47e-01aa-4d6c-90e7-c71c884882f3', multiLine = True)

In [3]:
df_telemetria

DataFrame[results: array<struct<datetime:string,id:bigint,machineid:bigint,pressure:double,rotate:double,vibration:double,volt:double>>]

In [4]:
df_telemetria = df_telemetria.withColumn("results", explode("results")) \
                    .withColumn("id", col("results")["id"].cast("integer")) \
                    .withColumn("datetime", col("results")["datetime"].cast("string")) \
                    .withColumn("machineid", col("results")["machineid"].cast("integer")) \
                    .withColumn("pressure", col("results")["pressure"].cast("double")) \
                    .withColumn("rotate", col("results")["rotate"].cast("double")) \
                    .withColumn("vibration", col("results")["sensovibration"].cast("double")) \
                    .withColumn("volt", col("results")["volt"].cast("double")) \
                    .drop("results") \
                    .orderBy("id")
df_telemetria2 = df_telemetria.toPandas()

AnalysisException: No such struct field sensovibration in datetime, id, machineid, pressure, rotate, vibration, volt